In [51]:
import cv2
import sqlite3
from matplotlib import pyplot as plt
import numpy as np

from visualize_model import Model
from database import blob_to_array, pair_id_to_image_ids

# 1. 3D mesh reconstruction from a set of images from the Gerrard Hall dataset.
Install Colmap and run the automatic reconstruction on the Gerrard Hall dataset

# 2. Analyze reconstructions using python
## 2.1. Run the notebook, using the Gerrard Hall reconstruction (0.5)
#### <span style='color:Green'> - Add the path to your reconstruction. Answer the questions at the end  </span>

In [52]:
# Add your path
reconstruction_path = "/home/arnau-marcos-almansa/workspace/MCV-C4/lab4/gerrard_hall/images/sparse/0"
database_path = "/home/arnau-marcos-almansa/workspace/MCV-C4/lab4/gerrard_hall/images/database.db"

#### Load an existing reconstruction and print its contents

In [53]:
model = Model()
# model.read_model(reconstruction_path, ext='.bin') # Should also work with .txt
model.read_model(reconstruction_path, ext='.bin')

In [54]:
images = model.images
cameras = model.cameras
points3D = model.points3D

In [55]:
print(f"Loaded {len(images)} images. This is the information available for one of them:")
print(images[1])
print(f"\nLoaded {len(cameras)} cameras. This is the information available for one of them:")
print(cameras[1])
print(f"\nLoaded {len(points3D)} 3D points. This is the information available for one of them:")
print(points3D[1])

Loaded 98 images. This is the information available for one of them:
Image(id=1, qvec=array([-0.14172196,  0.02182061,  0.96793132, -0.20627094]), tvec=array([0.95229568, 1.09044391, 4.43946955]), camera_id=1, name='IMG_2331.JPG', xys=array([[1729.23510742, 1121.40771484],
       [ 874.83270264, 2874.30810547],
       [2838.81811523, 2808.38208008],
       ...,
       [4822.82910156, 2396.75610352],
       [4268.22167969, 2416.02880859],
       [3265.3918457 , 3404.12133789]], shape=(10872, 2)), point3D_ids=array([   -1,    -1,    -1, ...,    -1, 67031,    -1], shape=(10872,)))

Loaded 1 cameras. This is the information available for one of them:
Camera(id=1, model='SIMPLE_RADIAL', width=5616, height=3744, params=array([ 3.82603137e+03,  2.80800000e+03,  1.87200000e+03, -6.97334946e-02]))

Loaded 72280 3D points. This is the information available for one of them:


KeyError: 1

#### Load the database

In [56]:
db = sqlite3.connect(database_path)

In [57]:
keypoints = dict(
        (image_id, blob_to_array(data, np.float32, (-1, 2)))
        for image_id, data in db.execute(
            "SELECT image_id, data FROM keypoints"))

In [58]:
print(f"Loaded keypoints from {len(keypoints)} images. These are the {len(keypoints[1])} keypoints for one of them:")
print(keypoints[1])

Loaded keypoints from 100 images. These are the 32616 keypoints for one of them:
[[ 1.7292351e+03  1.1214077e+03]
 [ 3.9607193e+02  3.3253399e-14]
 [ 2.4758311e+02  6.4772351e+02]
 ...
 [ 3.2653918e+03  3.4041213e+03]
 [ 4.7361712e+00 -5.1958436e-16]
 [ 1.5824780e+00  5.1334004e+00]]


In [59]:
matches = dict()
count_no_data = 0
for pair_id, data in db.execute("SELECT pair_id, data FROM matches"):
    if data is None:
        count_no_data += 1
    else:
        matches[pair_id_to_image_ids(pair_id)] = blob_to_array(data, np.uint32, (-1, 2))
print(f"Loaded {len(matches)} matches. {count_no_data}/{len(matches)+count_no_data} matches contained no data")

Loaded 1868 matches. 3082/4950 matches contained no data


In [60]:
# Temporal

dir(matches)
matches.__class__
matches.items()
matches.keys()

dict_keys([(1.0, 2), (1.0, 3), (1.0, 4), (1.0, 5), (1.0, 6), (1.0, 7), (1.0, 8), (1.0, 9), (1.0, 10), (1.0, 11), (1.0, 12), (1.0, 14), (1.0, 15), (1.0, 16), (1.0, 18), (1.0, 93), (1.0, 94), (1.0, 95), (1.0, 96), (1.0, 97), (1.0, 98), (1.0, 99), (1.0, 100), (2.0, 3), (2.0, 4), (2.0, 5), (2.0, 6), (2.0, 7), (2.0, 8), (2.0, 9), (2.0, 10), (2.0, 11), (2.0, 12), (2.0, 14), (2.0, 15), (2.0, 16), (2.0, 18), (2.0, 93), (2.0, 94), (2.0, 95), (2.0, 96), (2.0, 97), (2.0, 98), (2.0, 99), (2.0, 100), (3.0, 4), (3.0, 5), (3.0, 6), (3.0, 7), (3.0, 8), (3.0, 9), (3.0, 10), (3.0, 11), (3.0, 12), (3.0, 14), (3.0, 15), (3.0, 16), (3.0, 18), (3.0, 32), (3.0, 35), (3.0, 48), (3.0, 89), (3.0, 93), (3.0, 94), (3.0, 95), (3.0, 96), (3.0, 97), (3.0, 98), (3.0, 99), (3.0, 100), (4.0, 5), (4.0, 6), (4.0, 7), (4.0, 8), (4.0, 9), (4.0, 10), (4.0, 11), (4.0, 12), (4.0, 14), (4.0, 15), (4.0, 16), (4.0, 18), (4.0, 93), (4.0, 94), (4.0, 95), (4.0, 96), (4.0, 97), (4.0, 98), (4.0, 99), (4.0, 100), (5.0, 6), (5.0, 7), (

In [61]:
print("These are the matches between two images:")
# print(matches[1,2])
print(matches[1.0, 3])

These are the matches between two images:
[[     13272          0]
 [  16908295  102551425]
 [3356295801      32197]
 ...
 [      9418      11142]
 [     10672      11145]
 [      7907      11149]]


#### Visualize the point cloud and cameras

In [62]:
model.create_window()
model.add_points()
model.add_cameras(scale=0.25)
model.show()

#### <span style='color:Green'>  How many keypoints there are in total? </span> 

#### <span style='color:Green'>  How many 3D points originated from a keypoint in the first image? </span>


## 2.2 Plot the 3D points coloured according to the number of images and error. (0.5)

#### <span style='color:Green'> - Plot the 3D points coloured according to the **number of images** from which it originated. </span> Can you extract any conclusions from the visualization? 

In [ ]:
### TO DO 2.2

#### <span style='color:Green'> - Plot the 3D points coloured according to the **error**. </span> - What is this parameter? Can you extract any conclusions from the visualization?

In [ ]:
### TO DO 2.2

## 2.3 Plot the 3D points that correspond to a keypoint in the first image. Also plot the image with the keypoints (1.0)


In [ ]:
### TO DO 2.3

## 2.4 Create a visualization for the number of matches between all images. (1.0)
For example: https://seaborn.pydata.org/generated/seaborn.heatmap.html

In [ ]:
### TO DO 2.4

## 2.5 Visualize the keypoints and matches between the two images used in lab 3 using Colmap, how it compares to the results from lab 3? (1.0)
#### <span style='color:Green'> You can use the GUI to get the keypoints and matches and then visualize it here, following the same style as in lab 3 to get comparable results. </span>

In [ ]:
### TO DO 2.5

## 2.6 Triangulate and visualize the 3D points from the keypoints extracted using Colmap on the two images used in lab 3, how it compares to the results from lab 3? (1.0) 
#### <span style='color:Green'> - Use the triangulation from lab 3 to the get the 3D points and visualize them following the same style. </span>

In [ ]:
### TO DO 2.6

## 2.7 Visualize the sparse reconstruction using the 2 images from lab 3, and the complete CASTLE dataset. Comment on the differences between techniques and number of images used. (1.0)
#### <span style='color:Green'> - Use the reconstruction from Colmap to the get the 3D points and visualize them following the same style, using two images and the complete dataset. </span>

In [ ]:
### TO DO 2.7